In [4]:
import os
import numpy as np
from multiprocessing import  Pool
import random

In [8]:
num_threads=28
k_part=5
ori_data_path = '/home/angel00540/work_/work/cgh_2022/exp_0603/data_syllable/training/training_data/ori/'
aug_data_path = '/home/angel00540/work_/work/cgh_2022/exp_0603/data_syllable/training/training_data/aug/'
zpy_save_path='/home/angel00540/work_/work/cgh_2022/exp_0607/'
origin_list = []
aug_list = [[] for i in range(k_part)]
train_data=[[] for i in range(k_part)]
test_data=[[] for i in range(k_part)]
classes = []
aug_limit=999999
test_limit=999999
SEED = 44

In [9]:
classes=[]

def load_class():
    for _, dirs, _ in os.walk(os.path.abspath(ori_data_path)):
        break
    dirs = [int(x) for x in dirs]
    dirs.sort()
    dirs = [str(x) for x in dirs]
    return dirs

classes=load_class()
print(classes)

['0', '16']


In [10]:
def chunkIt(seq, num): #把一個list seq分成chunks of size num
    avg = len(seq) / float(num)
    out = []
    last = 0.0
    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    return out

In [11]:
def load_ori_list():
    print('Load ori list')
    folder_list=[[] for _ in range(k_part)]
    class_list=[]
    for n in classes:
        ori_path = ori_data_path + str(n)+'/'
        ori_n = 0
        tmp_list=[]
        for root2, _, files2 in os.walk(os.path.abspath(ori_path)):
            for file2 in files2:
                if('npy' in file2):
                    tmp_list.append(os.path.join(root2, file2))
                    ori_n = ori_n+1
        print('Class: ', n, ' files: ',ori_n)
        random.Random(SEED).shuffle(tmp_list)# 把tmp_list弄亂

        tmp_list = chunkIt(tmp_list, k_part) # 把tmp_list(代表某一個錯誤類別或是正確類別)分成k_part

        for k in range(k_part):
            print('part: ', k,' Class: ', n,' ori_n: ',len(tmp_list[k]))
            folder_list[k]=np.append(folder_list[k],tmp_list[k]) #folder_list裡面會有5個list，存有每個類別的5個part
            # folder_list[n]=folder_list[n]+tmp_list[n]
            
    return folder_list

origin_list=load_ori_list()
for n in range(k_part):
    print('part: ',n,' total_ori_n: ',len(origin_list[n]))

Load ori list
Class:  0  files:  108
part:  0  Class:  0  ori_n:  21
part:  1  Class:  0  ori_n:  22
part:  2  Class:  0  ori_n:  21
part:  3  Class:  0  ori_n:  22
part:  4  Class:  0  ori_n:  22
Class:  16  files:  446
part:  0  Class:  16  ori_n:  89
part:  1  Class:  16  ori_n:  89
part:  2  Class:  16  ori_n:  89
part:  3  Class:  16  ori_n:  89
part:  4  Class:  16  ori_n:  90
part:  0  total_ori_n:  110
part:  1  total_ori_n:  111
part:  2  total_ori_n:  110
part:  3  total_ori_n:  111
part:  4  total_ori_n:  112


In [12]:
len(origin_list)

5

In [13]:
print(origin_list[0][0])

/home/angel00540/work_/work/cgh_2022/exp_0603/data_syllable/training/training_data/ori/0/1_1463545338_03_07_2.npy


In [14]:
def load_aug_list():
    print('Load aug list')
    tmp_aug_list = [[] for i in range(k_part)]
    for i in range(k_part):
        tmp_list = [y[:-4] for y in origin_list[i]] #去除掉.npy
        tmp_list = [y[y.rfind('/')+1:] for y in tmp_list] #將.../training/training_data/ori/0/1_1463545338_03_07_2 轉成1_1463545338_03_07_2
        for root1, _, files1 in os.walk(os.path.abspath(aug_data_path)):
            for file1 in files1: # file1 like 1_1462500010_05_04_2_ts_.npy
                if('npy' in file1): 
                    myname = file1[:file1.rfind('_')] # myname like 1_1462500010_05_04_2_ts
                    myname = myname[:myname.rfind('_')] # myname like 1_1462500010_05_04_2
                    if(myname in tmp_list): #如果myname在這part的origin_list的話，就把此aug路徑也加到同part的aug_list
                        tmp_aug_list[i].append(os.path.join(root1, file1)) #origin_list[1]的aug都在aug_list[1]當中
    return tmp_aug_list
aug_list = load_aug_list()
for i in range(k_part):
    print('part: ',i,' aug_n: ',len(aug_list[i]))

Load aug list
part:  0  aug_n:  147
part:  1  aug_n:  154
part:  2  aug_n:  147
part:  3  aug_n:  154
part:  4  aug_n:  154


In [16]:
len(aug_list[0])

147

In [17]:
for i in range(k_part):
    print('part: ',i,' aug_n: ',len(aug_list[i]))

part:  0  aug_n:  147
part:  1  aug_n:  154
part:  2  aug_n:  147
part:  3  aug_n:  154
part:  4  aug_n:  154


In [11]:
def balance_aug_list():
    pass

In [12]:
def balance_test_list():
    pass

## marge_ori_aug

In [13]:
test_data=[[] for i in range(k_part)]
train_data=[[] for i in range(k_part)]

for select_part in range(k_part):
    for other_part in range(k_part):
        if select_part != other_part:
            train_data[select_part] = np.append(train_data[select_part] ,origin_list[other_part] )
            train_data[select_part]= np.append(train_data[select_part] , aug_list[other_part])
        else:
            test_data[select_part] = origin_list[select_part]
    print('part: ',select_part,' train_data: ',len(train_data[select_part]))
    print('part: ',select_part,' test_data: ',len(test_data[select_part]))


part:  0  train_data:  1255
part:  0  test_data:  160
part:  1  train_data:  1247
part:  1  test_data:  161
part:  2  train_data:  1253
part:  2  test_data:  162
part:  3  train_data:  1247
part:  3  test_data:  161
part:  4  train_data:  1246
part:  4  test_data:  162


In [14]:
def load_data(npy_name):
    lable = npy_name.split('/') #npy_name lik ..../training/training_data/ori/0/1_1463545338_03_07_2.npy'
    lable = classes.index(lable[-2]) #拿到label的資料夾
    data = np.load(npy_name[:-4]+'.npy', allow_pickle=True)
    return data, lable

In [15]:
(a , a_label) = load_data(train_data[0][100])
print(train_data[0][100])
print(a.shape,a_label)

/home/angel00540/work_/work/cgh_2022/exp_0607/training_data/ori/16/16_56554239_04_01_1.npy
(128, 128, 3) 1


In [16]:
def save_zpy():
    pool = Pool(num_threads)
    for n in range(k_part):
        print('Saving', zpy_save_path+'part_'+str(n)+'.zpy ...')
        x_train = []
        y_train = []
        x_test = []
        y_test = []

        pool_outputs = pool.map(load_data, train_data[n])
        for train,label in pool_outputs:
            x_train.append(train)
            y_train.append(label)

        pool_outputs = pool.map(load_data, test_data[n])
        for train,label in pool_outputs:
            x_test.append(train)
            y_test.append(label)

        print('x_train len: ', len(x_train))
        print('x_test len: ',len(x_test))

        np.savez(
            zpy_save_path+'part_'+str(n),
            x_train=np.asarray(x_train),
            y_train=np.asarray(y_train),
            x_test=np.asarray(x_test),
            y_test=np.asarray(y_test)
        )
    print('done')

save_zpy()

Saving /home/angel00540/work_/work/cgh_2022/exp_0607/part_0.zpy ...
x_train len:  1255
x_test len:  160
Saving /home/angel00540/work_/work/cgh_2022/exp_0607/part_1.zpy ...
x_train len:  1247
x_test len:  161
Saving /home/angel00540/work_/work/cgh_2022/exp_0607/part_2.zpy ...
x_train len:  1253
x_test len:  162
Saving /home/angel00540/work_/work/cgh_2022/exp_0607/part_3.zpy ...
x_train len:  1247
x_test len:  161
Saving /home/angel00540/work_/work/cgh_2022/exp_0607/part_4.zpy ...
x_train len:  1246
x_test len:  162
done


In [20]:
myArch = np.load(zpy_save_path+'part_0.npz')
x_train = myArch['x_train']
y_train = myArch['y_train']


In [28]:
x_train[0]

array([[[-20.723267 , -20.72233  , -17.848858 ],
        [-20.723267 , -20.720978 , -20.557343 ],
        [-20.709158 , -20.580164 , -18.179161 ],
        ...,
        [-10.051773 ,  -9.070483 ,  -7.2797503],
        [-11.543555 ,  -8.496667 ,  -6.9719405],
        [-10.681403 ,  -8.257528 ,  -7.3225017]],

       [[-20.723267 , -20.721983 , -17.230473 ],
        [-20.723267 , -20.720125 , -20.519484 ],
        [-20.705793 , -20.559395 , -17.63157  ],
        ...,
        [ -9.83373  ,  -8.3634205,  -7.8909063],
        [-11.325532 ,  -8.852437 ,  -7.8481927],
        [-10.46338  ,  -9.131693 ,  -7.896838 ]],

       [[-20.723267 , -20.721355 , -17.09755  ],
        [-20.723267 , -20.71859  , -20.512932 ],
        [-20.662853 , -20.580996 , -17.51406  ],
        ...,
        [ -7.6096563,  -6.7607684,  -6.1234956],
        [ -8.405311 ,  -7.0193205,  -6.5517406],
        [ -7.802183 ,  -7.128696 ,  -6.0640173]],

       ...,

       [[-20.723267 , -20.722946 , -19.946136 ],
        [-2

In [29]:
y_train[0]

0

In [24]:
train_data[0][0]

'/home/angel00540/work_/work/cgh_2022/exp_0607/training_data/ori/0/5_1399332403_05_05_1.npy'

In [25]:
test_x_train = np.load(train_data[0][0])

In [26]:
test_x_train.shape

(128, 128, 3)

In [27]:
test_x_train

array([[[-20.723267 , -20.72233  , -17.848858 ],
        [-20.723267 , -20.720978 , -20.557343 ],
        [-20.709158 , -20.580164 , -18.179161 ],
        ...,
        [-10.051773 ,  -9.070483 ,  -7.2797503],
        [-11.543555 ,  -8.496667 ,  -6.9719405],
        [-10.681403 ,  -8.257528 ,  -7.3225017]],

       [[-20.723267 , -20.721983 , -17.230473 ],
        [-20.723267 , -20.720125 , -20.519484 ],
        [-20.705793 , -20.559395 , -17.63157  ],
        ...,
        [ -9.83373  ,  -8.3634205,  -7.8909063],
        [-11.325532 ,  -8.852437 ,  -7.8481927],
        [-10.46338  ,  -9.131693 ,  -7.896838 ]],

       [[-20.723267 , -20.721355 , -17.09755  ],
        [-20.723267 , -20.71859  , -20.512932 ],
        [-20.662853 , -20.580996 , -17.51406  ],
        ...,
        [ -7.6096563,  -6.7607684,  -6.1234956],
        [ -8.405311 ,  -7.0193205,  -6.5517406],
        [ -7.802183 ,  -7.128696 ,  -6.0640173]],

       ...,

       [[-20.723267 , -20.722946 , -19.946136 ],
        [-2